In [1]:
import boto3
from pathlib import Path


In [2]:
# !pip install -U pystac-client
from pystac_client import Client

In [3]:
URL = 'https://stac.maap-project.org/'
cat = Client.open(URL)
cat

ID: stac-fastapi
Title: MAAP STAC API (dev)
Description: STAC API for the MAAP STAC system.
type: Catalog
"conformsTo: ['https://api.stacspec.org/v1.0.0-rc.1/item-search#filter', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#context', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#fields', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#sort', 'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/core', 'http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/features-filter', 'https://api.stacspec.org/v1.0.0-rc.1/item-search', 'https://api.stacspec.org/v1.0.0-rc.1/ogcapi-features', 'https://api.stacspec.org/v1.0.0-rc.1/core', 'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/geojson', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#filter:cql-text', 'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/oas30', 'https://api.stacspec.org/v1.0.0-rc.1/collections', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#filter:basic-cql', 'https://api.stacspec.org/v1.0.0-rc.1/item-search#query', 'http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/filter']"
https://raw.githubusercontent.com/radiantearth/stac-api-spec/v1.0.0-rc.1/fragments/context/json-schema/schema.json
ID: ESACCI_Biomass_L4_AGB_V3_100m_2018
"Title: ESA CCI above-ground biomass product level 4, year 2018"
"Description: This dataset comprises estimates of forest above-ground biomass for the years 2010, 2017 and 2018. They are derived from a combination of Earth observation data, depending on the year, from the Copernicus Sentinel-1 mission, Envisat’s ASAR instrument and JAXA’s Advanced Land Observing Satellite (ALOS-1 and ALOS-2), along with additional information from Earth observation sources. The data has been produced as part of the European Space Agency's (ESA's) Climate Change Initiative (CCI) programme by the Biomass CCI team."
type: Collection
"title: ESA CCI above-ground biomass product level 4, year 2018"


In [4]:
for collection in cat.get_all_collections():                                  
    print(collection.id)

ESACCI_Biomass_L4_AGB_V3_100m_2018
ESACCI_Biomass_L4_AGB_V3_100m_2017
ESACCI_Biomass_L4_AGB_V3_100m_2010
Landsat8_SurfaceReflectance
Global_PALSAR2_PALSAR_FNF
Global_Forest_Change_2000-2017
AFRISAR_DLR2
AfriSAR_UAVSAR_KZ
AfriSAR_UAVSAR_Ungeocoded_Covariance
AfriSAR_UAVSAR_Normalization_Area
AfriSAR_UAVSAR_Geocoded_SLC
AfriSAR_UAVSAR_Geocoded_Covariance
GlobCover_09
GlobCover_05_06
GEDI_CalVal_Field_Data
AfriSAR_UAVSAR_Coreg_SLC
GEDI_CalVal_Lidar_Data_Compressed
GEDI_CalVal_Lidar_Data
ALOS_PSR_RTC_HIGH
ABoVE_UAVSAR_PALSAR
AFRISAR_DLR
BIOSAR1
NASA_JPL_global_agb_mean_2020
icesat2-boreal
ESACCI_Biomass_L4_AGB_V4_100m_2017


In [12]:
s3 = boto3.resource('s3')
data_bucket = s3.Bucket("nasa-maap-data-store")
collections = []
for obj in data_bucket.objects.filter(Prefix="file-staging/nasa-map"):
    path = Path(obj.key)
    if obj.key.endswith(".tif"):
        continue
    collections.append(path.parts[2])
no_of_collections = len(list(set(collections)))
print(f"List of collections: {set(collections)}")
print(f"Number of collections in s3 bucket: {no_of_collections}")

List of collections: {'topo_a41_r9', 'GlobCover_09___2.3', 'AfriSAR_UAVSAR_Coreg_SLC___1', 'LVISF2___1', 'Global_Forest_Change_2000-2017___1.5', 'USER_SHARED_EX_1210___001', 'LVISF1B___1', 'Polarimetric_height_profile_1577___1', 'EnvStress___1', 'ATL03___004', 'SENTINEL-1A_DP_GRD_HIGH___1', 'ATL08___004', 'AfriSAR_UAVSAR_Geocoded_Covariance___1', 'AfriSAR_AGB_Maps_1681___1', 'AfriSAR_UAVSAR_SLC___2', 'GEDI_CalVal_Field_Data___1', 'Global_PALSAR2_PALSAR_Mosiac___1', 'PolInSAR_Canopy_Height_1589___1', 'GEDI_CalVal_Lidar_Data___2', 'AfriSAR_UAVSAR_Normalization_Area___1', 'ABLVIS2___1', 'ATL08_ARD-beta___001', 'GEDI_CalVal_Lidar_Data___1', 'AFLVIS2___001', 'GEDI01_B___001', 'GEDI02_A___002', 'AfriSAR_KingAir_B200_flight_tracks_Gabon___1', 'ATL08_001_user-added___0', 'AfriSAR_UAVSAR_Geocoded_SLC___1', 'AfriSAR_UAVSAR_Ungeocoded_Covariance___1', 'GEDI_CalVal_Field_USA_Sonoma___1', 'GEDI_CalVal_Lidar_USA_Sonoma___1', 'lvis_collection_level_2___2.0.1', 'GEDI_CalVal_Lidar_Data_Compressed___2',

In [14]:
size_map = {}
for collection in list(set(collections)):
    collection_size = 0
#     print(f"Processing collection: {collection}")
    for obj in data_bucket.objects.filter(Prefix=f"file-staging/nasa-map/{collection}"):
        collection_size+=obj.size
#     print(f"Collection size: {collection_size}")
    size_map[collection] = collection_size
print(size_map)

{'topo_a41_r9': 109985654, 'GlobCover_09___2.3': 12635, 'AfriSAR_UAVSAR_Coreg_SLC___1': 308088151748, 'LVISF2___1': 322688184792, 'Global_Forest_Change_2000-2017___1.5': 95199860180, 'USER_SHARED_EX_1210___001': 10123199, 'LVISF1B___1': 2433400098170, 'Polarimetric_height_profile_1577___1': 15134655664, 'EnvStress___1': 91770254, 'ATL03___004': 261251882255514, 'SENTINEL-1A_DP_GRD_HIGH___1': 1124610778, 'ATL08___004': 14601770249580, 'AfriSAR_UAVSAR_Geocoded_Covariance___1': 3000132108, 'AfriSAR_AGB_Maps_1681___1': 590104, 'AfriSAR_UAVSAR_SLC___2': 193408049996, 'GEDI_CalVal_Field_Data___1': 112911971, 'Global_PALSAR2_PALSAR_Mosiac___1': 10261220825591, 'PolInSAR_Canopy_Height_1589___1': 242233139, 'GEDI_CalVal_Lidar_Data___2': 3925040289975, 'AfriSAR_UAVSAR_Normalization_Area___1': 18335795, 'ABLVIS2___1': 111562109695, 'ATL08_ARD-beta___001': 3588897073543, 'GEDI_CalVal_Lidar_Data___1': 144494831842, 'AFLVIS2___001': 16356217403, 'GEDI01_B___001': 9823114468932, 'GEDI02_A___002': 720

In [39]:
all_s3_collection = [i.lower() for i in size_map]
s3_collection_string = ','.join(all_s3_collection)
stac_s3_intersection = []
stac_s3_map = {}
for collection in cat.get_all_collections():                                  
    if collection.id.lower() in s3_collection_string:
        stac_s3_intersection.append(collection.id)

In [40]:
stac_s3_intersection

['Landsat8_SurfaceReflectance',
 'Global_PALSAR2_PALSAR_FNF',
 'Global_Forest_Change_2000-2017',
 'AfriSAR_UAVSAR_KZ',
 'AfriSAR_UAVSAR_Ungeocoded_Covariance',
 'AfriSAR_UAVSAR_Normalization_Area',
 'AfriSAR_UAVSAR_Geocoded_SLC',
 'AfriSAR_UAVSAR_Geocoded_Covariance',
 'GlobCover_09',
 'GlobCover_05_06',
 'GEDI_CalVal_Field_Data',
 'AfriSAR_UAVSAR_Coreg_SLC',
 'GEDI_CalVal_Lidar_Data_Compressed',
 'GEDI_CalVal_Lidar_Data',
 'ALOS_PSR_RTC_HIGH',
 'ABoVE_UAVSAR_PALSAR',
 'icesat2-boreal']

In [41]:
s3_stac_renamed = [i+"___1" for i in stac_s3_intersection]
s3_stac_renamed

['Landsat8_SurfaceReflectance___1',
 'Global_PALSAR2_PALSAR_FNF___1',
 'Global_Forest_Change_2000-2017___1',
 'AfriSAR_UAVSAR_KZ___1',
 'AfriSAR_UAVSAR_Ungeocoded_Covariance___1',
 'AfriSAR_UAVSAR_Normalization_Area___1',
 'AfriSAR_UAVSAR_Geocoded_SLC___1',
 'AfriSAR_UAVSAR_Geocoded_Covariance___1',
 'GlobCover_09___1',
 'GlobCover_05_06___1',
 'GEDI_CalVal_Field_Data___1',
 'AfriSAR_UAVSAR_Coreg_SLC___1',
 'GEDI_CalVal_Lidar_Data_Compressed___1',
 'GEDI_CalVal_Lidar_Data___1',
 'ALOS_PSR_RTC_HIGH___1',
 'ABoVE_UAVSAR_PALSAR___1',
 'icesat2-boreal___1']

In [42]:
for coll in s3_stac_renamed:
    if coll in size_map.keys():
        print(f"{coll}--->{size_map[coll]*1e+9} GB")

Landsat8_SurfaceReflectance___1--->9.44541889768e+20 GB
Global_PALSAR2_PALSAR_FNF___1--->2.8370587656e+19 GB
AfriSAR_UAVSAR_KZ___1--->3.23509774e+17 GB
AfriSAR_UAVSAR_Ungeocoded_Covariance___1--->1.723097597e+18 GB
AfriSAR_UAVSAR_Normalization_Area___1--->1.8335795e+16 GB
AfriSAR_UAVSAR_Geocoded_SLC___1--->6.82527142e+18 GB
AfriSAR_UAVSAR_Geocoded_Covariance___1--->3.000132108e+18 GB
GEDI_CalVal_Field_Data___1--->1.12911971e+17 GB
AfriSAR_UAVSAR_Coreg_SLC___1--->3.08088151748e+20 GB
GEDI_CalVal_Lidar_Data___1--->1.44494831842e+20 GB
ALOS_PSR_RTC_HIGH___1--->5.71732666507e+20 GB
ABoVE_UAVSAR_PALSAR___1--->1.8799705287e+20 GB
